In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData, ForagerData
from os import sep as sep
import AnalysisFunctions as afuns

In [2]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [3]:
exp_num = 16
exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)

fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
# tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
# bdata = pd.read_csv(exp_path + sep + r'with food' + sep + r'blob analysis normalized by white paper' + sep + r'bdata_11_140720.csv')
# crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
ants = pd.read_csv(exp_path + sep + r'ants_list.csv')

In [4]:
ants

,ant_id,is_forager
0,1,False
1,1152,True
2,1165,False
3,139,False
4,142,False
5,148,False
6,153,True
7,171,False
8,173,False
9,179,False


In [5]:
foragers = [ant['ant_id'] for idx, ant in ants.iterrows() if ant['is_forager']==True]
foragers

[1152, 153, 180, 429, 454, 540, 72, 82, 232, 95, 535]

In [6]:
fdata.head()

,ant_id,feeding_start,feeding_end,food_source,crop_before,crop_after,feeding_size_intensity,feeding_size_ul
0,535,1,82,yellow,0.000000,453415.031250,453415.031250,9.780179
1,540,25,101,yellow,0.000000,347484.789673,347484.789673,8.780215
2,1152,69,140,yellow,0.000000,328380.593750,328380.593750,3.795152
3,82,120,172,yellow,0.000000,231418.015625,231418.015625,6.346243
4,540,156,193,yellow,347484.789673,350099.856009,2615.066336,0.066077


In [7]:
small_feedings_thres = 0.5  # ul
short_feedings_thres = 10  # frames

In [8]:
def get_forager_feedings(forager, fdata, small_feedings_thres=0, short_feedings_thres=0):
    forager_feedings = fdata[(fdata['ant_id']==forager) & (fdata['feeding_size_ul']>=small_feedings_thres) & 
                             ((fdata['feeding_end']-fdata['feeding_start'])>=short_feedings_thres)]
    red_feedings = forager_feedings[forager_feedings['food_source']=='red']
    yellow_feedings = forager_feedings[forager_feedings['food_source']=='yellow']
    return len(forager_feedings), len(red_feedings), len(yellow_feedings)

In [12]:
num_feedings = []
num_red_feedings = []
num_yellow_feedings = []
forager_id = []
for forager in foragers:
    num_feedings_f, num_red_feedings_f, num_yellow_feedings_f = get_forager_feedings(forager, fdata, small_feedings_thres=small_feedings_thres)
    forager_id.append(forager)
    num_feedings.append(num_feedings_f)
    num_red_feedings.append(num_red_feedings_f)
    num_yellow_feedings.append(num_yellow_feedings_f)
foragers_feedings_data = pd.DataFrame({'forager_id':forager_id, 'num_feedings': num_feedings, 'num_red_feedings': num_red_feedings, 'num_yellow_feedings': num_yellow_feedings})
foragers_feedings_data

,forager_id,num_feedings,num_red_feedings,num_yellow_feedings
0,1152,6,2,4
1,153,39,33,6
2,180,1,0,1
3,429,2,0,2
4,454,7,7,0
5,540,18,10,8
6,72,4,4,0
7,82,7,6,1
8,232,7,7,0
9,95,6,6,0


In [19]:
tasters_threshold = 6
specialist_ratio = 0.67

In [20]:
tasters = foragers_feedings_data.forager_id[foragers_feedings_data.num_feedings<tasters_threshold]
tasters

2    180
3    429
6     72
Name: forager_id, dtype: int64

In [21]:
yellow_foragers = foragers_feedings_data.forager_id[(foragers_feedings_data.num_yellow_feedings/foragers_feedings_data.num_feedings>specialist_ratio) & (~foragers_feedings_data.forager_id.isin(tasters.values))]
yellow_foragers

Series([], Name: forager_id, dtype: int64)

In [22]:
red_foragers = foragers_feedings_data.forager_id[(foragers_feedings_data.num_red_feedings/foragers_feedings_data.num_feedings>specialist_ratio) & (~foragers_feedings_data.forager_id.isin(tasters.values))]
red_foragers

1     153
4     454
7      82
8     232
9      95
10    535
Name: forager_id, dtype: int64

In [23]:
mix_foragers = foragers_feedings_data.forager_id[(~foragers_feedings_data.forager_id.isin(red_foragers.values)) & (~foragers_feedings_data.forager_id.isin(yellow_foragers.values)) & (~foragers_feedings_data.forager_id.isin(tasters.values))]
mix_foragers

0    1152
5     540
Name: forager_id, dtype: int64

In [ ]:
foragers_feedings_data['forager_type']=np.nan
foragers_feedings_data.forager_type[foragers_feedings_data['forager_id'].isin(tasters.values)]='taster'
foragers_feedings_data.forager_type[foragers_feedings_data['forager_id'].isin(yellow_foragers.values)]='protein_forager'
foragers_feedings_data.forager_type[foragers_feedings_data['forager_id'].isin(red_foragers.values)]='sugar_forager'
foragers_feedings_data.forager_type[foragers_feedings_data['forager_id'].isin(mix_foragers.values)]='mix_forager'
foragers_feedings_data

In [ ]:
foragers_feedings_data[foragers_feedings_data.forager_type != 'taster']

In [ ]:
foragers_feedings_data.to_csv(exp_path + sep + f'forager_types_tasters{tasters_threshold}_specialists{specialist_ratio}.csv', index=False)